In [1]:
# imports
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

2025-05-04 18:37:12.552881: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-04 18:37:12.558648: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-04 18:37:12.572433: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746398232.593543 2831358 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746398232.599943 2831358 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746398232.618813 2831358 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
# load and prep data
X, _ = load_digits(return_X_y=True)
X = X / X.max()  # normalize data

In [3]:
# pca
pca = PCA(n_components=64)
X_pca_reduced = pca.fit_transform(X)
X_pca_recon = pca.inverse_transform(X_pca_reduced)
pca_mse = np.mean((X - X_pca_recon) ** 2)

In [4]:
# autoencoder setup
input_layer = Input(shape=(64,))
encoded = Dense(32, activation='relu')(input_layer)
encoded = Dense(2, activation='linear')(encoded)
decoded = Dense(32, activation='relu')(encoded)
decoded = Dense(64, activation='sigmoid')(decoded)

autoencoder = Model(inputs=input_layer, outputs=decoded)
encoder = Model(inputs=input_layer, outputs=encoded)
autoencoder.compile(optimizer=Adam(1e-3), loss='mse')
autoencoder.fit(X, X, epochs=50, batch_size=128, shuffle=True, verbose=0)

# reconstruction
X_auto_recon = autoencoder.predict(X)
auto_mse = np.mean((X - X_auto_recon) ** 2)

2025-05-04 18:37:22.939445: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


KeyboardInterrupt: 

In [ ]:
# Print comparison
print(f"PCA Reconstruction MSE: {pca_mse:.4f}")
print(f"Autoencoder Reconstruction MSE: {auto_mse:.4f}")